In [1]:
import tensorflow as tf

# Load the saved model


mobilenetv1 = tf.keras.models.load_model('/kaggle/input/mobilenetv1-model/kaggle/working/mobilenetV1_97.keras')
vgg19 = tf.keras.models.load_model('/kaggle/input/vgg19-model/kaggle/working/vgg19_keras_9167.keras')
vgg16 = tf.keras.models.load_model('/kaggle/input/vgg16-model/kaggle/working/vgg16_keras_95.h5')
mobilenetv2 =  tf.keras.models.load_model('/kaggle/input/mobilenetv2-model/MobileNetV2/best_mobilenetv2_model.keras')
l

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the test ImageDataGenerator without shuffling
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the test generator with shuffle=False
test_generator = test_datagen.flow_from_directory(
    "/kaggle/input/dry-fruit-classification-split/Split_Dataset/test",                # Replace with your test directory path
    target_size=(224, 224),  # Replace with your target size
    batch_size=32,           # Replace with your desired batch size
    class_mode='categorical',# Adjust according to your labels
    shuffle=False            # Ensure the data is not shuffled
)


Found 2308 images belonging to 4 classes.


In [9]:
import pandas as pd
df = pd.DataFrame({
    "Image_No": [ i for i in range( 1 , 2308+1 )]
})
df.head(2)

,Image_No
0,1
1,2


In [3]:
# test_loss, test_acc = mobilenetv1.evaluate(test_generator)
# print( test_loss , test_acc )

softmax_probs_mobilenetv1 = mobilenetv1.predict( test_generator )
df[["MobileNetV1_1","MobileNetV1_2","MobileNetV1_3","MobileNetV1_4"]] = softmax_probs_mobilenetv1


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 15s 181ms/step


In [4]:
softmax_probs_vgg19 =  vgg19.predict( test_generator )
df[["vgg19_1","vgg19_2","vgg19_3","vgg19_4"]] = softmax_probs_vgg19


73/73 ━━━━━━━━━━━━━━━━━━━━ 16s 172ms/step


In [5]:
softmax_probs_vgg16 = vgg16.predict( test_generator )
df[["vgg16_1","vgg16_2","vgg16_3","vgg16_4"]] = softmax_probs_vgg16

I0000 00:00:1724493246.345475     106 service.cc:145] XLA service 0x561ccf078f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724493246.345544     106 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1724493246.345549     106 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 2/73 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step 

I0000 00:00:1724493258.637758     106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


73/73 ━━━━━━━━━━━━━━━━━━━━ 24s 155ms/step


In [6]:
softmax_probs_mobilenetv2 = mobilenetv2.predict( test_generator )
df[["mobilenetv2_1","mobilenetv2_2","mobilenetv2_3","mobilenetv2_4"]] = softmax_probs_mobilenetv2

73/73 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step


In [23]:
########################### APPEND THE ACTUAL CLASS ########################
df['Actual'] = test_generator.classes

In [24]:
X = df.drop("Actual" , axis  =  1)
y = df["Actual"]

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
from sklearn.ensemble import RandomForestClassifier
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)
print("-------------------------RANDOM FOREST CLASSIFIER_---------------------------")
print( "Accuracy:" , rf_classifier.score( X_test , y_test ) )
print( classification_report( y_test , rf_classifier.predict( X_test )) )

-------------------------RANDOM FOREST CLASSIFIER_---------------------------
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       123
           1       1.00      1.00      1.00       107
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00       112

    accuracy                           1.00       462
   macro avg       1.00      1.00      1.00       462
weighted avg       1.00      1.00      1.00       462



In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize the Logistic Regression classifier
log_reg_classifier = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
log_reg_classifier.fit(X_train, y_train)

print("-------------------------LOGISTIC REGRESSION CLASSIFIER---------------------------")

# Evaluate the model
print("Accuracy:", log_reg_classifier.score(X_test, y_test))
print(classification_report(y_test, log_reg_classifier.predict(X_test)))


-------------------------LOGISTIC REGRESSION CLASSIFIER---------------------------
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       123
           1       1.00      1.00      1.00       107
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00       112

    accuracy                           1.00       462
   macro avg       1.00      1.00      1.00       462
weighted avg       1.00      1.00      1.00       462



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Initialize the Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the model
decision_tree_classifier.fit(X_train, y_train)

print("-------------------------DECISION TREE CLASSIFIER---------------------------")

# Evaluate the model
print("Accuracy:", decision_tree_classifier.score(X_test, y_test))
print(classification_report(y_test, decision_tree_classifier.predict(X_test)))


-------------------------DECISION TREE CLASSIFIER---------------------------
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       123
           1       1.00      1.00      1.00       107
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00       112

    accuracy                           1.00       462
   macro avg       1.00      1.00      1.00       462
weighted avg       1.00      1.00      1.00       462

